# Pre-Process London Crime datasets (Street, Outcome and London-Boroughs)
Street dataset pre-process before sending to Datawarehouse
1. London Crime Street Dataset
2. London Crime Outcome Dataset
***

## Common activities
### Import libraries 

In [1]:
# import all the libraries that require for project
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os

### Declare globals

In [2]:
# define paths
INPUT_PATH = "C://SHU/ADMP/Assessment_02/londoncrimedw_project/input_csv/"
OUTPUT_PATH = "C://SHU/ADMP/Assessment_02/londoncrimedw_project/output_csv/"

# create variables to store the datasets name you want to use
STREET_FILENAME = "street_london.csv"
OUTCOME_FILENAME = "outcome_london.csv"
BOROUGH_FILENAME = "london-boroughs-data.csv"

### Declare and define functions 

In [3]:
class FILESIZE:
    SMALL = 1
    LARGE = 2
    
def read_csv_file(fileType, fileName):
    if (fileType == FILESIZE.SMALL):
        df_ret = pd.read_csv(fileName)
    else:
        # read the large csv file with specified chunksize of 10 million records
        df_chunk = pd.read_csv(fileName, chunksize=1000000)

        # append each chunk df here
        chunk_list = []   

        # each chunk is in df format
        for chunk in df_chunk:    
            # once the data filtering is done, append the chunk to list
            chunk_list.append(chunk)

        # concat the list into dataframe 
        df_ret = pd.concat(chunk_list)
        
    return df_ret

### Read CSV files and store them in respective data frames

In [4]:
# set a working directory to the location where input dataset csv file exist
os.chdir(INPUT_PATH)

# load dataframe with street data from CSV file 
street_df = read_csv_file(FILESIZE.LARGE, STREET_FILENAME)

# load dataframe with outcome data from CSV file 
outcome_df = read_csv_file(FILESIZE.LARGE, OUTCOME_FILENAME)

# store london borough data into dataframe
london_borough_df = read_csv_file(FILESIZE.SMALL, BOROUGH_FILENAME)

## Start Pre-Processing for London Crime Street Dataset

### Handle NULL/Missing Values

In [5]:
# 01. Crime_ID: Remove null records from the street data frame.
street_df = street_df[street_df.Crime_ID.isnull() == False]

# 02. Longitude, Latitude: Fill missing values with 0
street_df.Longitude.fillna(0, inplace=True)
street_df.Latitude.fillna(0, inplace=True)

# 03. LSOA_code, LSOA_name: Fill missing values with 'Not Available' text
street_df.LSOA_code.fillna('Not Available', inplace=True)
street_df.LSOA_name.fillna('Not Available', inplace=True)

# 04. Last_outcome_category: Fill missing values with 'Not Available' text
street_df.Last_outcome_category.fillna('Not Available', inplace=True)

# 05. Context: Drop it as all fields are empty here. Hence, no significance
street_df = street_df.drop(['Context'], axis=1)

### Delete columns which are of not much significance
'Reported_by' column is not useful and hence drop it

In [6]:
# drop Reported_by column from the dataset 
street_df = street_df.drop(['Reported_by'], axis=1)

### Derive new column for Borough_name from LSOA_name

In [7]:
# remove last 5 characters from the LSOA_name (4 code and 1 space)
street_df['Borough_name'] = street_df.LSOA_name[(street_df.LSOA_name.isnull()==False)].str[:-5]

### Remove duplication for Crime_ID feature

In [8]:
# Remove all the duplicate records except first instance of it
street_df = street_df.drop_duplicates(subset='Crime_ID', keep='first')

### Split columns from one to two

Split Month column to two columns named, Year and Month.

In [9]:
# Check if the month column have any null value 
number_of_null_months = street_df.Month[street_df.Month.isnull() == True].size

#  Split  Month column to Year and Month only if their is no null value present inside Month
if(number_of_null_months == 0):
    # create Year column inside street dataframe and populate year data from the Month field
    street_df['Year'] = street_df['Month'].str[: 4]
    
    # update Month column with just a Month number
    street_df['Month'] = street_df['Month'].str[-2: ]    

## Integrate "London Borough CSV" with "Street Crime CSV"
***
### Integration key used to merge between two datasets is 'Borough_name'

In [10]:
# Merge london_borough with the street data to get relevant records only
street_df = pd.merge(left=street_df, right=london_borough_df, left_on='Borough_name', right_on='Name')

# Remove un-necessary columns after merging action
street_df = street_df.drop(['Id', 'Name'], axis=1)

# Get unique london-boroghs numbers
street_df.Borough_name.unique().size

33

## Start Pre-Processing for London Crime Outcome Dataset

### Do sorting on month basis
Outcome dataset sorting on the Month basis is require to bring latest updated outcome on the top for each crime ids and rest will lie beneath. This is important step because only the latest records are going to retain and all previous/historical outcome update would be deleted.

In [11]:
# Sort the outcome data frame with Month column in descending order to get the latest crime id entry on top
outcome_df = outcome_df.sort_values(by='Month', ascending=False)

# Test if the latest crime id is appearing on the top 
# outcome_df[outcome_df.Crime_ID == ('3b60aed0ce6c29f63a00e44822492dcdc419b68a0974e53e6884359dc2aec1aa')].sort_values(by='Month', ascending=False).head(2)

### Remove duplicate Crime IDs so that only the recent outcome update will be availble in the dataset

In [12]:
# Remove all the duplicate records except first instance of it
outcome_df = outcome_df.drop_duplicates(subset='Crime_ID', keep='first')

### Rename 'Outcome_type' column to 'Latest_Outcome_type' for better readability

In [13]:
# Rename the Outcome_type to Latest_outcome_type
outcome_df.rename(columns = {'Outcome_type':'Latest_Outcome_type'}, inplace = True)

### Keep only the required features inside the outcome dataset

In [14]:
# Need only two columns from the dataset and hence extract the required ones
outcome_df = outcome_df[['Crime_ID','Latest_Outcome_type']]

## Integrate "London Borough CSV" with "Outcome CSV"
***
### Integration key used to merge between two dataset is 'Crime_ID'

In [15]:
# Do the left outer joining so that all street data will be retained along with the joined outcome data
street_df = pd.merge(left=street_df, right=outcome_df, on='Crime_ID', how='left')

### Replace all empty/null 'Latest_outcome_type' with the 'Last_outcome_category' values

In [16]:
# replace null values with 'Last_outcome_category' text
street_df.Latest_Outcome_type.fillna(street_df['Last_outcome_category'], inplace=True)

### Remove "Last_outcome_category" and retain the "Latest outcome category" only

In [17]:
# Drop Last_outcome_category
street_df = street_df.drop(['Last_outcome_category'], axis=1)

### Rename the "Crime_ID" to "Crime_hashcode"

In [18]:
street_df.rename(columns = {'Crime_ID':'Crime_hashcode'}, inplace = True)

## Save processed dataframe to csv file in the output path location

In [19]:
street_df.to_csv(OUTPUT_PATH+"street-staging-data.csv", sep=',', encoding='utf-8', index=None, header = True)